In [1]:
from pykrx import stock
import pandas as pd
from datetime import datetime
import pickle
import warnings
warnings.filterwarnings('ignore')

# 빈 테이블 만들기

In [2]:
listColumns = ['stock_code','period','category1','category2','시가총액','상장주식수','총자산','현금','부채','지배자산',
              '매출액Y','매출원가Y','매출총이익Y','판관비Y','영업이익Y','당기순이익Y','지배순이익Y','영업활동현금흐름Y',
              '매출액Q','매출원가Q','매출총이익Q','판관비Q','영업이익Q','당기순이익Q','지배순이익Q','영업활동현금흐름Q',
              '매출액','매출원가','매출총이익','판관비','영업이익','당기순이익','지배순이익','영업활동현금흐름']
def makeDataframeBlank(dateBase : datetime):
    # kospi_list = stock.get_market_ticker_list(date, market='KOSPI')
    # kosdaq_list = stock.get_market_ticker_list(date, market='KOSDAQ')
    market_cap_kospi = stock.get_market_cap_by_ticker(date=dateBase, market='KOSPI')
    market_cap_kosdaq = stock.get_market_cap_by_ticker(date=dateBase, market='KOSDAQ')
    dfBlank = pd.DataFrame(columns = listColumns)
    dfBlank['stock_code']= list(market_cap_kospi.index) + list(market_cap_kosdaq.index)
    dfBlank['period'] = definePeriod(dateBase = dateBase)
    dfBlank['category1'] = ['kospi']*len(market_cap_kospi) + ['kosdaq']*len(market_cap_kosdaq)
    dfBlank['시가총액'] = list(market_cap_kospi['시가총액']) + list(market_cap_kosdaq['시가총액'])
    dfBlank['거래량'] = list(market_cap_kospi['거래량']) + list(market_cap_kosdaq['거래량'])
    dfBlank['거래대금'] = list(market_cap_kospi['거래대금']) + list(market_cap_kosdaq['거래대금'])
    dfBlank['상장주식수'] = list(market_cap_kospi['상장주식수']) + list(market_cap_kosdaq['상장주식수'])
    return dfBlank

In [3]:
def definePeriod(dateBase : datetime):
    if dateBase.month == 4:
        period = str(dateBase.year-1) + '12'
    elif dateBase.month == 5:
        period = str(dateBase.year) + '03'
    elif dateBase.month == 8:
        period = str(dateBase.year) + '06'
    elif dateBase.month == 11:
        period = str(dateBase.year) + '09'
    else:
        print("check dateBase!!!")
    return period

In [74]:
start = datetime(2019,9,1)
end = datetime(2021,11,16)
listPeriod = []
for year in range(start.year, end.year+1):
    for date in [datetime(year,4,1), datetime(year,5,15), datetime(year,8,15), datetime(year,11,15)]:
        if (date > start)&(date < end):
            listPeriod.append(date)
print(listPeriod)
dfBlankAll = pd.DataFrame()
for dateBase in listPeriod:
    dfBlankTemp = makeDataframeBlank(dateBase=dateBase)
    dfBlankAll = pd.concat([dfBlankAll,dfBlankTemp],axis=0)
dfBlankAll.reset_index(drop=True,inplace=True)

[datetime.datetime(2019, 11, 15, 0, 0), datetime.datetime(2020, 4, 1, 0, 0), datetime.datetime(2020, 5, 15, 0, 0), datetime.datetime(2020, 8, 15, 0, 0), datetime.datetime(2020, 11, 15, 0, 0), datetime.datetime(2021, 4, 1, 0, 0), datetime.datetime(2021, 5, 15, 0, 0), datetime.datetime(2021, 8, 15, 0, 0), datetime.datetime(2021, 11, 15, 0, 0)]


# DartAPI 자료 활용하여 값 넣기

## 출처 값을 그대로 넣기

In [7]:
value_dict = {'총자산':'ifrs-full_Assets', '현금':'ifrs-full_CashAndCashEquivalents',
              '부채':'ifrs-full_Liabilities','지배자산':'ifrs-full_EquityAttributableToOwnersOfParent',
             '매출액':'ifrs-full_Revenue', '매출원가':'ifrs-full_CostOfSales', 
              '매출총이익':'ifrs-full_GrossProfit', '판관비':'dart_TotalSellingGeneralAdministrativeExpenses',
              '영업이익':'dart_OperatingIncomeLoss',
#               '계속영업이익':'ifrs-full_ProfitLossFromContinuingOperations',
              '당기순이익':'ifrs-full_ProfitLoss', '지배순이익':'ifrs-full_ProfitLossAttributableToOwnersOfParent',
             '영업활동현금흐름':'ifrs-full_CashFlowsFromUsedInOperatingActivities'}

In [75]:
listCategory = ['총자산','현금','부채','지배자산','매출액','매출원가','매출총이익','판관비','영업이익','당기순이익','지배순이익','영업활동현금흐름']
# listPeriodStr = ['202109','202106','202103','202012','202009','202006','202003','201912', '201909']
listPeriodStr = list(set(dfBlankAll.period))
listPeriodStr.sort(reverse=True)

In [31]:
def makeSeriesValueByCategory(category : str):
    cases_temp = dfFinancePeriod[dfFinancePeriod.account_id == value_dict[category]][['stock_code','period','thstrm_amount']]
    cases_temp.drop_duplicates(['stock_code','period'],keep='first',inplace=True)
    dfBlankAll_temp = dfBlankAll[['stock_code','period']]
    seriesValue = pd.merge(dfBlankAll_temp,cases_temp,how='left',on=['stock_code','period'])['thstrm_amount']
    return seriesValue

In [76]:
for period in listPeriodStr:
    with open(f'Raw_Finance/dartAPI/{period}.pickle','rb') as handle:
        dfFinancePeriod = pickle.load(handle)
    dfFinancePeriod['period'] = period
    for nameCategory in listCategory:
    #     dfBlankAll.loc[dfBlankAll.period==period,nameCategory]=makeSeriesValueByCategory(nameCategory)
        dfBlankAll[nameCategory].fillna(makeSeriesValueByCategory(nameCategory), inplace=True)

In [77]:
for category in listCategory:
    dfBlankAll[category] = pd.to_numeric(dfBlankAll[category])

In [78]:
dfBlankAll
# .dropna(subset=['총자산'])

,stock_code,period,category1,category2,시가총액,상장주식수,총자산,현금,부채,지배자산,...,매출액,매출원가,매출총이익,판관비,영업이익,당기순이익,지배순이익,영업활동현금흐름,거래량,거래대금
0,005930,201909,kospi,NaN,320577322935000,5969782550,3.533860e+14,2.660499e+13,8.994374e+13,2.554034e+14,...,6.200347e+13,3.999389e+13,2.200958e+13,1.423169e+13,7.777892e+12,6.287664e+12,6.105039e+12,2.566578e+13,9703360,518314735700
1,000660,201909,kospi,NaN,62025801498000,728002365,6.367175e+13,1.203143e+12,1.540552e+13,4.825397e+13,...,6.838766e+12,5.023127e+12,1.815639e+12,1.343078e+12,4.725610e+11,4.954840e+11,4.932010e+11,3.899615e+12,2394138,203013766200
2,005935,201909,kospi,NaN,35466416770000,822886700,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1492581,64033499600
3,035420,201909,kospi,NaN,28595124032500,164813395,1.180672e+13,3.254602e+12,5.369200e+12,5.681910e+12,...,1.664815e+12,NaN,NaN,NaN,2.020860e+11,8.526805e+10,1.254284e+11,8.184173e+11,886846,154997769000
4,005380,201909,kospi,NaN,26815357468500,213668187,1.916699e+14,9.208985e+12,1.156447e+14,6.984067e+13,...,2.696885e+13,2.254047e+13,4.428386e+12,4.049866e+12,NaN,NaN,4.269110e+11,1.320577e+12,469109,58621093500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21406,335870,202109,kosdaq,NaN,6739200000,3120000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,49444,106904310
21407,323210,202109,kosdaq,NaN,6640000000,3200000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6656,13702160
21408,021045,202109,kosdaq,NaN,6363699000,4242466,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4622,6909665
21409,347140,202109,kosdaq,NaN,6348990000,3009000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,19849,41562950


## 출처값으로 Q, Y 값 넣기

In [ ]:
202012
-201912

In [73]:
listPeriodStr[0][-2:]

'09'

In [80]:
for i in range(len(listPeriodStr)-4):
    for category in listCategory[4:]:
        dfPeriod = dfBlankAll[dfBlankAll.period == listPeriodStr[i]][['stock_code','period',category]]
        if listPeriodStr[i][-2:] != '12':
            dfPeriodMinus4Q = dfBlankAll[dfBlankAll.period == listPeriodStr[i+4]][['stock_code',category]].rename(columns={category:category+'4Q'})
            dfPeriodLastYear = dfBlankAll[dfBlankAll.period == (str(int(listPeriodStr[i][:4])-1)+'12')][['stock_code',category]].rename(columns={category:category+'LY'})
            dfPeriod = pd.merge(dfPeriod,dfPeriodLastYear,how='left',on=['stock_code'])
            dfPeriod = pd.merge(dfPeriod,dfPeriodMinus4Q,how='left',on=['stock_code'])
            dfPeriod[f'{category}Y'] = dfPeriod[category] + dfPeriod[f'{category}LY'] - dfPeriod[f'{category}4Q']
        else:
            dfPeriod[f'{category}Y'] = dfPeriod[category]
        dfPeriodMinus1Q = dfBlankAll[dfBlankAll.period == listPeriodStr[i+1]][['stock_code',category]].rename(columns={category:category+'1Q'})
        dfPeriod = pd.merge(dfPeriod,dfPeriodMinus1Q,how='left',on=['stock_code'])
        dfPeriod[f'{category}Q'] = dfPeriod[category] - dfPeriod[f'{category}1Q']
        
        dfBlankAll_temp = dfBlankAll[['stock_code','period']]
        dfPeriod_temp = dfPeriod[['stock_code','period',f'{category}Y']]
        seriesValue = pd.merge(dfBlankAll_temp,dfPeriod_temp,how='left',on=['stock_code','period'])[f'{category}Y']
        dfBlankAll[f'{category}Y'].fillna(seriesValue, inplace=True)

        dfPeriod_temp = dfPeriod[['stock_code','period',f'{category}Q']]
        seriesValue = pd.merge(dfBlankAll_temp,dfPeriod_temp,how='left',on=['stock_code','period'])[f'{category}Q']
        dfBlankAll[f'{category}Q'].fillna(seriesValue, inplace=True)

In [92]:
dfAll = dfBlankAll[['stock_code','period','총자산','현금','부채','지배자산','매출액Y','매출원가Y',
                    '매출총이익Y','판관비Y','영업이익Y','당기순이익Y','지배순이익Y','영업활동현금흐름Y']]
dfAll = dfAll.rename(columns = {'매출액Y':'매출액','매출원가Y':'매출원가','매출총이익Y':'매출총이익','판관비Y':'판관비',
                      '영업이익Y':'영업이익','당기순이익Y':'당기순이익','지배순이익Y':'지배순이익','영업활동현금흐름Y':'영업활동현금흐름'})

In [46]:
dfAll = dfBlankAll[['stock_code','period','총자산','현금','부채','지배자산','매출액','매출원가','매출총이익','판관비','영업이익','당기순이익','지배순이익','영업활동현금흐름']]

In [95]:
dfAll['period'] = pd.to_numeric(dfAll['period'])

In [96]:
with open('Raw_Finance/finance_dartapi.pickle','wb') as handle:
    pickle.dump(dfAll, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [45]:
with open('Raw_Finance/finance_dartapi.pickle','rb') as handle:
    cases = pickle.load(handle)
cases

EOFError: Ran out of input

In [399]:
dfBlankAll[dfBlankAll.period == listPeriodStr[i]]['매출액Y']

21208    NaN
21209    NaN
21210    NaN
21211    NaN
21212    NaN
        ... 
23670    NaN
23671    NaN
23672    NaN
23673    NaN
23674    NaN
Name: 매출액Y, Length: 2467, dtype: object

In [398]:
dfBlankAll[dfBlankAll.period == listPeriodStr[i]]['매출액Y'] = 1

In [351]:
dfBlankAll[nameCategory].fillna(makeSeriesValueByCategory(nameCategory), inplace=True)

str

In [ ]:
df1 = df[df.period == '202009'][['값']].rename(columns=)
df2 = df[df.period == '201909'][['값']].rename(columns=)
df3 = df[df.period == '201912'][['값']].rename(columns=)
df4 = df[df.period == '202006'][['값']].rename(columns=)
df_year = df1+df3-df2
df_year = pd.merge([df1,df3,df2],how='left')
df[df.period == '202009'][['값Y']] = df_year['값Y']
df_quarter = df1-df4

In [298]:
nameValue = '당기순이익'
cases_temp = cases[cases.account_id == value_dict[nameValue]][['stock_code','period','thstrm_amount']]
dfBlankAll_temp = dfBlankAll[['stock_code','period']]
result = pd.merge(dfBlankAll_temp,cases_temp,how='left',on=['stock_code','period'])['thstrm_amount']
# dfBlankAll.loc[dfBlankAll.period==period,nameValue]=result

In [303]:
cases_temp.drop_duplicates(['stock_code','period'],keep='first',inplace=True)

In [304]:
pd.merge(dfBlankAll_temp,cases_temp,how='left',on=['stock_code','period'])
# ['thstrm_amount']

,stock_code,period,thstrm_amount
0,005930,202009,9360693000000
1,000660,202009,1077862000000
2,207940,202009,56081371017
3,051910,202009,570387000000
4,005935,202009,NaN
...,...,...,...
2360,080440,202009,NaN
2361,323210,202009,NaN
2362,158310,202009,-1923145379
2363,347140,202009,NaN


In [228]:
cases_temp = cases[cases.account_id == value_dict[nameValue2]][['stock_code','period','thstrm_amount']]
dfBlankAll_temp = dfBlankAll[['stock_code','period']]
result2 = pd.merge(dfBlankAll_temp,cases_temp,how='left',on=['stock_code','period'])['thstrm_amount']

In [246]:
# result.fillna(result2,inplace=True)
dfBlankAll.loc[dfBlankAll.period==period,nameValue] = result

In [262]:
type(result2)

pandas.core.series.Series

In [254]:
dfBlankAll.영업이익.fillna(result2,inplace=True)
a.d.fillna(a.c, inplace=True)

In [295]:
dfFinancePeriod.loc[(dfFinancePeriod.stock_code == '121890')&(dfFinancePeriod.account_id == 'ifrs-full_Assets')]

,rcept_no,reprt_code,bsns_year,corp_code,sj_div,sj_nm,account_id,account_nm,account_detail,thstrm_nm,thstrm_amount,frmtrm_nm,frmtrm_amount,ord,thstrm_add_amount,frmtrm_q_nm,frmtrm_q_amount,frmtrm_add_amount,stock_code,period
241134,20201106000355,11014,2020,00604815,BS,재무상태표,ifrs-full_Assets,자산총계,-,제 20 기 3분기말,15675254286,제 19 기말,27231033195,18,NaN,NaN,NaN,NaN,121890,202009


In [308]:
dfBlankAll.tail(20)#16329

,stock_code,period,category1,category2,시가총액,상장주식수,총자산,현금,부채,지배자산,...,지배순이익Q,영업활동현금흐름Q,매출액,매출원가,매출총이익,판관비,영업이익,당기순이익,지배순이익,영업활동현금흐름
18380,021045,202009,kosdaq,NaN,8124322390,4242466,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18381,331380,202009,kosdaq,NaN,8040050000,4010000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18382,331520,202009,kosdaq,NaN,7837600000,3880000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18383,08537M,202009,kosdaq,NaN,7724019600,715187,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18384,329560,202009,kosdaq,NaN,7680750000,3850000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18385,033600,202009,kosdaq,NaN,7308230437,40828103,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18386,311270,202009,kosdaq,NaN,7254000000,3600000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18387,349720,202009,kosdaq,NaN,7203800000,3620000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18388,323280,202009,kosdaq,NaN,7167725000,3505000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18389,121890,202009,kosdaq,NaN,7101198800,9609200,15675254286,2085842099,6512095399,NaN,...,NaN,NaN,5546319063,4695377967,850941096,766772275,84168821,-331310128,NaN,3012934356


In [313]:
dfBlankAll.isnull().sum()#16329

stock_code        0
period            0
category1         0
category2     18400
시가총액              0
상장주식수             0
총자산            8265
현금             8264
부채             8255
지배자산          10441
매출액Y          18400
매출원가Y         18400
매출총이익Y        18400
판관비Y          18400
영업이익Y         18400
당기순이익Y        18400
지배순이익Y        18400
영업활동현금흐름Y     18400
매출액Q          18400
매출원가Q         18400
매출총이익Q        18400
판관비Q          18400
영업이익Q         18400
당기순이익Q        18400
지배순이익Q        18400
영업활동현금흐름Q     18400
매출액            8427
매출원가           8793
매출총이익          8861
판관비            8818
영업이익           8230
당기순이익          8297
지배순이익         11377
영업활동현금흐름       8341
dtype: int64

In [ ]:
a = pd.DataFrame({'c':[1,2,3],'d':[np.nan,2,1]})
a.d.fillna(a.c, inplace=True)

In [156]:
len(cases[(cases.stock_code == '005930')&(cases.period == period)&(cases.account_id == value_dict[nameValue])]['thstrm_amount'])
# .values[0]

1

In [137]:
# stock_code = '005930'
nameValue = '총자산'

In [114]:
listStockCode = [stock_code for stock_code, periodTemp in dfBlankAll.index if periodTemp == period]

In [159]:
for stock_code in listStockCode:
#     for nameValue in value_dict.keys():
    value = cases[(cases.stock_code == stock_code)&(cases.period == period)&(cases.account_id == value_dict[nameValue])]['thstrm_amount']
    if len(value) == 1:
        dfBlankAll.at[(stock_code,period), nameValue] = value.values[0]

In [160]:
dfBlankAll

,,category1,category2,시가총액,상장주식수,총자산,현금,부채,지배자산,매출액Y,매출원가Y,...,영업활동현금흐름Q,매출액,매출원가,매출총이익,판관비,영업이익,당기순이익,지배순이익,영업활동현금흐름,계속영업이익
stock_code,period,,,,,,,,,,,,,,,,,,,,,
005930,201812,kospi,NaN,268938703877500,5969782550,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
000660,201812,kospi,NaN,55764981159000,728002365,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
005935,201812,kospi,NaN,29953075880000,822886700,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
051910,201812,kospi,NaN,26154463081500,70592343,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
005380,201812,kospi,NaN,25747016533500,213668187,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
080440,202009,kosdaq,NaN,6636328080,13325960,375788742000000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
323210,202009,kosdaq,NaN,6416000000,3200000,375788742000000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
158310,202009,kosdaq,NaN,6096179210,12071642,29622362749,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [135]:
a = cases.loc[('000660', '202009', )]
a[a.sj_div == 'CIS']
# set(list(a.sj_div))

,rcept_no,reprt_code,bsns_year,corp_code,sj_div,sj_nm,account_nm,account_detail,thstrm_nm,thstrm_amount,frmtrm_nm,frmtrm_amount,ord,thstrm_add_amount,frmtrm_q_nm,frmtrm_q_amount,frmtrm_add_amount
account_id,,,,,,,,,,,,,,,,,
ifrs-full_Revenue,20201116001609,11014,2020,00164779,CIS,포괄손익계산서,매출액,-,제 73 기 3분기,8128767000000,NaN,NaN,0,23934193000000,제 72 기 3분기,6838766000000,20063622000000
ifrs-full_CostOfSales,20201116001609,11014,2020,00164779,CIS,포괄손익계산서,매출원가,-,제 73 기 3분기,5285510000000,NaN,NaN,1,15554658000000,제 72 기 3분기,5023127000000,13550971000000
ifrs-full_GrossProfit,20201116001609,11014,2020,00164779,CIS,포괄손익계산서,매출총이익,-,제 73 기 3분기,2843257000000,NaN,NaN,2,8379535000000,제 72 기 3분기,1815639000000,6512651000000
dart_TotalSellingGeneralAdministrativeExpenses,20201116001609,11014,2020,00164779,CIS,포괄손익계산서,판매비와관리비,-,제 73 기 3분기,1543561000000,NaN,NaN,3,4332823000000,제 72 기 3분기,1343078000000,4035970000000
dart_OperatingIncomeLoss,20201116001609,11014,2020,00164779,CIS,포괄손익계산서,영업이익,-,제 73 기 3분기,1299696000000,NaN,NaN,4,4046712000000,제 72 기 3분기,472561000000,2476681000000
ifrs-full_FinanceIncome,20201116001609,11014,2020,00164779,CIS,포괄손익계산서,금융수익,-,제 73 기 3분기,346708000000,NaN,NaN,5,1139390000000,제 72 기 3분기,416962000000,1305369000000
ifrs-full_FinanceCosts,20201116001609,11014,2020,00164779,CIS,포괄손익계산서,금융비용,-,제 73 기 3분기,301542000000,NaN,NaN,6,1130276000000,제 72 기 3분기,389229000000,1113882000000
-표준계정코드 미사용-,20201116001609,11014,2020,00164779,CIS,포괄손익계산서,지분법투자 관련 손익,-,제 73 기 3분기,-3459000000,NaN,NaN,7,12719000000,제 72 기 3분기,4469000000,14522000000
dart_OtherGains,20201116001609,11014,2020,00164779,CIS,포괄손익계산서,기타영업외수익,-,제 73 기 3분기,23568000000,NaN,NaN,8,61631000000,제 72 기 3분기,31440000000,80822000000


In [100]:
dfBlankAll.at[('005930','201812'),'시가총액']

268938703877500

In [143]:
cases.at[(stock_code,period)+(value_dict[nameValue],)]['thstrm_amount'][0]

TypeError: _get_value() got multiple values for argument 'takeable'

In [104]:
dfBlankAll.at[(stock_code,period),nameValue]

'375788742000000'

In [18]:
set([name[-3:] for name in list(cases[cases.sj_div == 'IS']['thstrm_nm'])])

{'3분기'}

In [40]:
cases['period']=202103
cases.set_index(keys=['stock_code','period','account_id'],inplace=True)

In [43]:
index_value = dfBlankAll.index[0]
nameValue = '총자산'

In [49]:
a = cases.index[0]

In [64]:
int(cases.at[a]['rcept_no'][0])

20201116002078

In [67]:
index_value

('005930', '201812')

In [66]:
cases.at[index_value+(value_dict[nameValue],)]['thstrm_amount'][0]

KeyError: '201812'

In [173]:
cases_temp = cases[cases.account_id == value_dict[nameValue]][['stock_code','period','thstrm_amount']]
dfBlankAll_temp = dfBlankAll[['stock_code','period']]
result = pd.merge(dfBlankAll_temp,cases_temp,how='left',on=['stock_code','period'])['thstrm_amount']
dfBlankAll.loc[dfBlankAll.period=='202009','총자산']=result

,stock_code,period,thstrm_amount
16,035000,202009,446783022237
285,134380,202009,137792592231
538,285130,202009,2194366970444
723,005180,202009,776151411159
855,003920,202009,1020318646475
...,...,...,...
332289,032960,202009,60200445866
332538,082800,202009,259737196799
332754,139670,202009,17528114834
332869,134060,202009,21087338234


In [219]:
dfBlankAll_temp = dfBlankAll[['stock_code','period']]
result = pd.merge(dfBlankAll_temp,cases_temp,how='left',on=['stock_code','period'])['thstrm_amount']
dfBlankAll.loc[dfBlankAll.period=='202009','총자산']=result

In [184]:
result = pd.merge(dfBlankAll_temp,cases_temp,how='left',on=['stock_code','period'])['thstrm_amount']
dfBlankAll[nameValue]=result

In [221]:
dfBlankAll.loc[dfBlankAll.period=='202009','총자산']=result

In [223]:
dfBlankAll.tail(100)

,stock_code,period,category1,category2,시가총액,상장주식수,총자산,현금,부채,지배자산,...,지배순이익Q,영업활동현금흐름Q,매출액,매출원가,매출총이익,판관비,영업이익,당기순이익,지배순이익,영업활동현금흐름
18300,084180,202009,kosdaq,NaN,24986571750,15864490,50683609301,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18301,127160,202009,kosdaq,NaN,24394380435,66833919,120985975207,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18302,900080,202009,kosdaq,NaN,24380639821,123759593,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18303,134580,202009,kosdaq,NaN,24075000000,11250000,26368871100,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18304,020400,202009,kosdaq,NaN,23860555650,2607711,103731603196,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18395,080440,202009,kosdaq,NaN,6636328080,13325960,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18396,323210,202009,kosdaq,NaN,6416000000,3200000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18397,158310,202009,kosdaq,NaN,6096179210,12071642,29622362749,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18398,347140,202009,kosdaq,NaN,6093225000,3009000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [190]:
np.nan

nan

In [198]:
a = pd.DataFrame({'c':[1,2,3],'d':[np.nan,2,1]})
a.d.fillna(a.c, inplace=True)
a

,c,d
0,1,1.0
1,2,2.0
2,3,1.0


In [197]:
a.d.fillna(a.c, inplace=True)
a

,c,d
0,1,1.0
1,2,2.0
2,3,1.0
